In [1]:
###Pandas Libraries
import pandas as pd
import datetime
import time

#Dash Libraries
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

In [78]:
#*********                     FUNCTION RUNS ALL OF THE DATA MANIPULATION CODE
## TIMER FUNCTION
start = time.time()

mt_beds = 13 #Current Number of Beds Available in Maternity
ds_beds = 4  #Current Number of Beds Available in Delivery Suite

## Bring in Transfers data, update dates and remove uneeded columns
df = pd.read_csv('transfers.csv')
df = df.rename(columns = {'Start_Bed_Dttm':'Start', 'End_Bed_Dttm':'End'})
df = df.drop(columns=['Transfers', 'Bed_Code', 'Full Flow Code', 'Current Flow Code', '1st Level','PASID'])
date_cols = ['Start', 'End']
df[date_cols] = df[date_cols].apply(pd.to_datetime, format='%d/%m/%y %H:%M', errors='raise')
#df[date_cols] = df[date_cols].apply(lambda x: x.dt.strftime('%d/%m/%Y %H:%M:%S'))

#MATERNITY
## Make new dataframe patients entering Maternity Ward
mt_in_df= pd.DataFrame()
mt_in_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'Start']
mt_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
mt_out_df= pd.DataFrame()
mt_out_df['Event'] = df.loc[df['Ward_Code'] == 'MT', 'End']
mt_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
mt_count_df = pd.concat([mt_in_df, mt_out_df])
mt_count_df.sort_values('Event', ascending = True, inplace=True)
mt_count_df['MT_Patients'] = mt_count_df['Patient_Change'].cumsum()
mt_count_df['Bed_Available'] = mt_count_df['MT_Patients'] <= mt_beds
mt_count_df.dropna(inplace = True)

# DELIVERY SUITE(DS)
## Make new dataframe patients entering room
ds_in_df = pd.DataFrame()
ds_in_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'Start']
ds_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
ds_out_df= pd.DataFrame()
ds_out_df['Event'] = df.loc[df['Ward_Code'] == 'DS', 'End']
ds_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
ds_count_df = pd.concat([ds_in_df, ds_out_df])
ds_count_df.sort_values('Event', ascending = True, inplace=True)
ds_count_df['DS_Patients'] = ds_count_df['Patient_Change'].cumsum()
ds_count_df['Bed_Available'] = ds_count_df['DS_Patients'] <= mt_beds
ds_count_df.dropna(inplace = True)

# CLEAN DATA TO PRODUCE IDEAL PATIENT TRANSFERS
ideal_df = pd.DataFrame(df)
#CA, CL & HE Codes are converted to MT
ideal_df.loc[(ideal_df['Ward_Code'] == 'CA') | (ideal_df['Ward_Code'] == 'CL') | (ideal_df['Ward_Code'] == 'HE') , 'Ward_Code'] = 'MT' #Ward Code CA = MT
## Drop DPS from the new dataset
index_names = ideal_df.loc[(ideal_df['Ward_Code'] == 'DPS') | ((ideal_df['Ward_Code'] == 'ON'))].index
ideal_df.drop(index_names, inplace = True)
## Groupby the event code and with ward to summarise and finds any duplicates
events = ideal_df.groupby(['Link', 'Ward_Code'])
#events.get_group(('ADE-272865','MT'))
#get the min start date and max end date from the grouped results
ideal_df['min_start'] = events['Start'].transform('min')
ideal_df['max_end'] = events['End'].transform('max')
#Drop the Original start & end dates & rename the new calculated start & end
ideal_df.drop(columns=['Start', 'End'], inplace = True)
ideal_df.rename(columns= {'min_start':'Start', 'max_end': 'End'}, inplace = True)
ideal_df.sort_values(['Link', 'Ward_Code'], ascending = True, inplace = True)
ideal_df.drop_duplicates(keep='first', inplace = True)
#checks that there are no longer any duplicates
#ideal_df.loc[ideal_df.duplicated(keep=False), :]

# CALCULATE LENGTH OF STAY
ideal_df['LOS'] = (ideal_df['End'] - ideal_df['Start'])
ideal_df['LOS_hrs'] = ((ideal_df['End'] - ideal_df['Start']).dt.days * 24) + ((ideal_df['End'] - ideal_df['Start']).dt.seconds / (60*60))

# EXTRACT EVENTS DATA AND PUT IN THE IDEALISED DATA TABLE
events_df = pd.read_csv('exception_dates.csv')
#events_df.drop(columns=['Unnamed: 2'], inplace = True)
events_df.rename(columns = {'DATE':'Date', 'EVENT':'Event'}, inplace=True)
events_df['Date'] = events_df['Date'].apply(pd.to_datetime, format='%d/%m/%y', errors='raise')
events_df['Date'] = events_df['Date'].dt.date
events_df.sort_values(['Event', 'Date'])
## Combine all labels with the same date into the same field
grouped_events_df = events_df.groupby(['Date'])['Event'].apply(','.join).reset_index()
grouped_events_df.sort_values('Event', ascending=True)
## Delete erroneous Event Data
grouped_events_df = grouped_events_df.drop(grouped_events_df[grouped_events_df.Event == 'SCHOOL,SCHOOL'].index)
grouped_events_df = grouped_events_df.drop(grouped_events_df[grouped_events_df.Event == 'PUBLIC,SCHOOL,SCHOOL'].index)
grouped_events_df['Count'] = 1
## Merge data from the events table back into the Idealised data table
ideal_df['Date'] = ideal_df['Start'].dt.date
ideal_merged_df = pd.merge(grouped_events_df, ideal_df, how='right', on='Date')
ideal_merged_df.fillna('NORMAL', inplace = True)
ideal_df = ideal_merged_df

#Represent Patient process flow by counting number of instances of a flow path
##Remove unneeded data and sort by patient event
flow_df = ideal_df.loc[:, :]#['Link', 'Ward_Code', 'Start']]
flow_df['Count'] = 1
flow_df.sort_values(['Link', 'Start'], inplace=True)

#function for returning the previous flow code
def prev_flow(df, n):
    for i in range(n, 0, -1):
        df.Prev_Flow = np.where(df.Link == df.Link.shift(i), 
                 df.Prev_Flow + '-' + df.Ward_Code.shift(i), df.Prev_Flow)
    return df.Prev_Flow
flow_df['Prev_Flow'] = 'IN'
flow_df['Prev_Flow'] = prev_flow(flow_df, 6)
#flow_df.loc[flow_df.Link == 'ADE-300013']

#JOIN ALL OF THE GROUP FLOW CODES TOGETHER
flow_df['Full_Flow_Code'] = flow_df.groupby(['Link'])['Ward_Code'].transform(lambda x: '-'.join(x))
flow_df['Full_Flow_Code'] ='IN' + '-' + flow_df.Full_Flow_Code
#flow_df.groupby('Full_Flow_Code').Full_Flow_Code.count()

#DAY OF THE WEEK
ideal_df = flow_df
ideal_df['Day_of_Week'] = flow_df.Start.dt.day_name()
#MONTH NAME
ideal_df['Month'] = flow_df.Start.dt.month_name()
#YEAR
ideal_df['Year'] = flow_df.Start.dt.year
#HOUR OF DAY
ideal_df['Hour_of_Day'] = flow_df.Start.dt.hour

#INTERARRIVAL TIME
ideal_df = ideal_df.sort_values('Start')
interarrival_time = ideal_df.loc[flow_df.Prev_Flow == 'IN', 'Start'] - ideal_df.loc[flow_df.Prev_Flow == 'IN', 'Start'].shift(1)
ideal_df['Interarrival_Time'] = pd.to_numeric(interarrival_time.dt.days*24 + interarrival_time.dt.seconds / (60*60))
arrival_df = ideal_df[(ideal_df.Prev_Flow == 'IN') & (ideal_df.Interarrival_Time.notnull())]



#MATERNITY IDEAL FLOW
## Make new dataframe patients entering Maternity Ward
mt_in_df= pd.DataFrame()
mt_in_df['Event'] = ideal_df.loc[df['Ward_Code'] == 'MT', 'Start']
mt_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
mt_out_df= pd.DataFrame()
mt_out_df['Event'] = ideal_df.loc[df['Ward_Code'] == 'MT', 'End']
mt_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
mt_count_df = pd.concat([mt_in_df, mt_out_df])
mt_count_df.sort_values('Event', ascending = True, inplace=True)
mt_count_df['MT_Patients'] = mt_count_df['Patient_Change'].cumsum()
mt_count_df['Bed_Available'] = mt_count_df['MT_Patients'] <= mt_beds
mt_count_df.dropna(inplace = True)

# DELIVERY SUITE(DS) IDEAL FLOW
## Make new dataframe patients entering room
ds_in_df = pd.DataFrame()
ds_in_df['Event'] = ideal_df.loc[df['Ward_Code'] == 'DS', 'Start']
ds_in_df['Patient_Change'] = 1
## make new dataframe with patients leaving room
ds_out_df= pd.DataFrame()
ds_out_df['Event'] = ideal_df.loc[df['Ward_Code'] == 'DS', 'End']
ds_out_df['Patient_Change'] = -1
## Concatenate the two datasets and sort by date to determin current patinet occupancies
ds_count_df = pd.concat([ds_in_df, ds_out_df])
ds_count_df.sort_values('Event', ascending = True, inplace=True)
ds_count_df['DS_Patients'] = ds_count_df['Patient_Change'].cumsum()
ds_count_df['Bed_Available'] = ds_count_df['DS_Patients'] <= mt_beds
ds_count_df.dropna(inplace = True)









end = time.time()
print(str(round(end - start,1)) + ' seconds to run code.')

2.0 seconds to run code.


In [57]:
search_for = ['SCHOOL','B4EASTER']

base = r'^{}'
expr = '(?=.*{})'
search_exp = base.format(''.join(expr.format(w) for w in search_for))

len(ideal_df.loc[ideal_df.Event.str.contains(search_exp)].index)

39

In [42]:
grouped_events_df.groupby(['Event'], as_index=False)['Count'].count().sort_values('Count', ascending = False)

,Event,Count
13,SCHOOL,912
16,"SCHOOL,BEFOREXMAS",78
14,"SCHOOL,AFTERXMAS",63
1,B4EASTER,56
8,"PUBLIC,SCHOOL",47
2,BEFOREXMAS,46
3,"BEFOREXMAS,SCHOOL",44
5,PUBLIC,34
0,"AFTERXMAS,SCHOOL",30
7,"PUBLIC,EASTER",23


In [44]:
events_df.groupby('Event').count()

,Date
Event,
AFTERXMAS,96
B4EASTER,70
BEFOREXMAS,168
EASTER,54
PUBLIC,159
SCHOOL,1273
XMAS,30


In [76]:
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

#app = dash.Dash(__name__) # USE THIS IIF RUNNING ON SERVER
app = JupyterDash(__name__) # USE THIS IF RUNNING ON JUPYTER

variability_selection = [
            {'label': 'After Christmas', 'value': 'AFTERXMAS'},
            {'label': 'Week Before Easter', 'value': 'B4EASTER'},
            {'label': '2 Weeks Before Christmas', 'value': 'BEFOREXMAS'},
            {'label': 'Easter', 'value': 'EASTER'},
            {'label': 'No Event', 'value': 'NORMAL'},
            {'label': 'Public Holiday', 'value': 'PUBLIC'},
            {'label': 'School Holidays', 'value': 'SCHOOL'},
            {'label': 'Christmas', 'value': 'XMAS'},
                ]

app.layout = html.Div([
    
    html.Div([
        html.Pre(children= "Variability Analysis",
                style={"text-align": "center", "font-size": "100%", "color":"black"})
    ]),
    
    dcc.Graph(id='the_graph'),
    
    dcc.Dropdown(
        id='my_dropdown',
        options=variability_selection,
        optionHeight=35,
        value='',
        disabled=False,
        multi=False,
        searchable=True,
        search_value='',
        placeholder='Please select Varabilities for Scenario #1 ...',
        clearable=True
        #style={'width':"100%"},
        #className='select_box',
        #persistence=True,
        #persistence_type='memory' #'session','local' 
                ),
    
    dcc.Dropdown(
        id='my_dropdown2',
        options=variability_selection,
        optionHeight=35,
        value='',
        disabled=False,
        multi=True,
        searchable=True,
        search_value='',
        placeholder='Please select Varabilities for Scenario #2 ...',
        clearable=True
        #style={'width':"100%"},
        #className='select_box',
        #persistence=True,
        #persistence_type='memory' #'session','local' 
                )
])

@app.callback(
    Output(component_id ='the_graph', component_property='figure'), 
    [Input(component_id ='my_dropdown', component_property='value'),
     Input(component_id ='my_dropdown2', component_property='value')
    ])
def update_graph(my_dropdown, my_dropdown2):
    plot_df = ideal_df
    ## Converts list into a search strings expression
    base = r'^{}'
    expr = '(?=.*{})'
    search_exp = base.format(''.join(expr.format(w) for w in my_dropdown))
    search_exp2 = base.format(''.join(expr.format(w) for w in my_dropdown2))
    ## Filter table based of selected
    plot_df1 = plot_df.loc[plot_df.Event.str.contains(search_exp),:]
    plot_df2 = plot_df.loc[plot_df.Event.str.contains(search_exp2),:]
    
    ### Set up the two traces
    trace0 = go.Histogram(x = plot_df1.Interarrival_Time, histnorm='probability', xbins.size = 2)
    trace1 = go.Histogram(x = plot_df2.Interarrival_Time, histnorm='probability', xbins.size = 2)
    
    data = [trace0, trace1]
    fig = go.Figure(data = data)
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.75)
    
    
    #plot_df = plot_df.loc[ideal_df.Event.str.contains(search_exp),:]  #Plotly Express Single Histogram
    #fig = px.histogram(plot_df, x="Interarrival_Time", nbins=50)  # Plotly Express figure for Single histogram
    return fig

#app.run_server(debug=True)  # USE THIS IF RUNNING ON SERVER
app.run_server(mode='jupyterlab') # USE THIS IF RUNNING ON JUPYTER

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-76-caf161f72a30>, line 85)

In [6]:
######KEEP AS WORKING TEMPLATE
#import dash  # USE THIS IF RUNNING ON SERVER
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash # USE THIS IF RUNNING ON JUPYTER
import numpy as np

#app = dash.Dash(__name__) # USE THIS IIF RUNNING ON SERVER
app = JupyterDash(__name__) # USE THIS IF RUNNING ON JUPYTER

app.layout = html.Div([
    
    html.Div([
        html.Pre(children= " Length of Stay Histogram Plot",
                style={"text-align": "center", "font-size": "100%", "color":"black"})
    ]),
    
    dcc.Graph(id='the_graph'),
    
    dcc.Dropdown(
        id='my_dropdown',
        options=[
            {'label': 'Maternity', 'value': 'MT'},
            {'label': 'Delivery Suite', 'value': 'DS'},
            {'label': 'ICU', 'value': 'ICU'},
                ],
                 optionHeight=35,
                 value='MT',
                 disabled=False,
                 multi=False,
                 searchable=True,
                 search_value='',
                 placeholder='Please select... ',
                 clearable=True
                 #style={'width':"100%"},
                 #className='select_box',
                 #persistence=True,
                 #persistence_type='memory' #'session','local' 
                )
])

@app.callback(
    Output(component_id ='the_graph', component_property='figure'), 
    [Input(component_id ='my_dropdown', component_property='value')])
def update_graph(my_dropdown):
    plot_df = ideal_df
    plot_df = plot_df.loc[plot_df['Ward_Code'] == my_dropdown, :]
    fig = px.histogram(plot_df, x="LOS_hrs", nbins=50)
    return fig

#app.run_server(debug=True)  # USE THIS IF RUNNING ON SERVER
app.run_server(mode='jupyterlab') # USE THIS IF RUNNING ON JUPYTER

In [149]:
arrival_df

,Date,Event,Link,Ward_Code,Start,End,LOS,LOS_hrs,Count,Prev_Flow,Full_Flow_Code,Day_of_Week,Month,Year,Hour_of_Day,Interarrival_Time
5,2013-06-27,NaN,ADE-282775,MT,2013-06-27 16:00:00,2013-07-01 11:05:00,3 days 19:05:00,91.083333,1,IN,IN-MT,Thursday,June,2013,16,200.0
4,2013-06-27,NaN,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00,81.516667,1,IN,IN-MT,Thursday,June,2013,23,474.0
9,2013-06-28,NaN,ADE-282886,MT,2013-06-28 06:30:00,2013-07-02 10:09:00,4 days 03:39:00,99.650000,1,IN,IN-MT,Friday,June,2013,6,396.0
8,2013-06-28,NaN,ADE-273071,MT,2013-06-28 12:27:00,2013-07-02 11:05:00,3 days 22:38:00,94.633333,1,IN,IN-MT,Friday,June,2013,12,357.0
7,2013-06-28,NaN,ADE-272854,MT,2013-06-28 21:12:00,2013-07-01 10:37:00,2 days 13:25:00,61.416667,1,IN,IN-MT,Friday,June,2013,21,525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8161,2020-07-27,NaN,ADE-506202,MT,2020-07-27 13:43:00,2020-07-29 15:58:00,2 days 02:15:00,50.250000,1,IN,IN-MT,Monday,July,2020,13,2722.0
8159,2020-07-27,NaN,ADE-501132,MT,2020-07-27 22:32:00,2020-07-31 10:04:00,3 days 11:32:00,83.533333,1,IN,IN-MT,Monday,July,2020,22,529.0
8158,2020-07-28,NaN,ADE-510775,MT,2020-07-28 12:14:00,2020-07-28 17:15:00,0 days 05:01:00,5.016667,1,IN,IN-MT,Tuesday,July,2020,12,822.0
8157,2020-07-28,NaN,ADE-501113,MT,2020-07-28 15:29:00,2020-08-01 10:01:00,3 days 18:32:00,90.533333,1,IN,IN-MT,Tuesday,July,2020,15,195.0


In [156]:
ideal_df

,Date,Event,Link,Ward_Code,Start,End,LOS,LOS_hrs,Count,Prev_Flow,Full_Flow_Code,Day_of_Week,Month,Year,Hour_of_Day,Interarrival_Time
3,2013-06-27,NORMAL,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00,21.6667,1,IN,IN-DS-MT,Thursday,June,2013,12,NaN
5,2013-06-27,NORMAL,ADE-282775,MT,2013-06-27 16:00:00,2013-07-01 11:05:00,3 days 19:05:00,91.0833,1,IN,IN-MT,Thursday,June,2013,16,200.0
4,2013-06-27,NORMAL,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00,81.5167,1,IN,IN-MT,Thursday,June,2013,23,474.0
9,2013-06-28,NORMAL,ADE-282886,MT,2013-06-28 06:30:00,2013-07-02 10:09:00,4 days 03:39:00,99.65,1,IN,IN-MT,Friday,June,2013,6,396.0
6,2013-06-28,NORMAL,ADE-271206,MT,2013-06-28 10:20:00,2013-07-03 09:59:00,4 days 23:39:00,119.65,1,IN-DS,IN-DS-MT,Friday,June,2013,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8161,2020-07-27,NORMAL,ADE-506202,MT,2020-07-27 13:43:00,2020-07-29 15:58:00,2 days 02:15:00,50.25,1,IN,IN-MT,Monday,July,2020,13,2722.0
8159,2020-07-27,NORMAL,ADE-501132,MT,2020-07-27 22:32:00,2020-07-31 10:04:00,3 days 11:32:00,83.5333,1,IN,IN-MT,Monday,July,2020,22,529.0
8158,2020-07-28,NORMAL,ADE-510775,MT,2020-07-28 12:14:00,2020-07-28 17:15:00,0 days 05:01:00,5.01667,1,IN,IN-MT,Tuesday,July,2020,12,822.0
8157,2020-07-28,NORMAL,ADE-501113,MT,2020-07-28 15:29:00,2020-08-01 10:01:00,3 days 18:32:00,90.5333,1,IN,IN-MT,Tuesday,July,2020,15,195.0


# Maternity Service Capacity - Model, Simulation & Analysis
## Objective 1
**Model current Maternity Service activity using process mapping and queuing network analysis**
1. Data Required
2. Profile - Patient Process Flow - Historical
3. Profile - Patient Process Flow - Idealised
4. Profile - Length of Stay
5. Profile - Arrival Rate
7. Variability Analysis - Events of Significance
8. Variability Analysis - Patient Process Flow
9. Variability Analysis - Length of Stay
10. Variability Analysis - Arrival Rate


## Objective 2
**Determine current capacity requirements using simulation and mathematical modeling**
1. Current Capacity Requirements - Historical Data
2. Current Capacity Requirements - Idealised Historical Data
3. Current Capacity Requirements - Simulated Model
4. Results Analysis - Accuracy with Idealised Historical Data

## Objective 3
**Determine future capacity requirements at 5 and 10 years based on population demographics projections data.**
1. Capacity Requirements - 5 year forecast
2. Capacity Requirements - 10 year forecast

***

## Objective 1
### 1.1. Data Required
#### 1.1.1 Historical Data
7 years of patient transfers data provided by hospital.
Data will provide the foundation of the mathematical model & data analysis

In [6]:
df

,Link,Ward_Code,Start,End
0,ADE-363169,MT,2016-02-22 05:30:00,2016-02-27 11:45:00
1,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00
2,ADE-461781,ICU,2019-05-09 18:17:00,2019-05-10 13:24:00
3,ADE-501083,DS,2020-06-20 20:00:00,2020-06-22 03:38:00
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00
...,...,...,...,...
8921,ADE-348899,MT,2016-02-15 14:27:00,2016-02-19 10:15:00
8922,ADE-491342,DS,2019-12-15 20:00:00,2019-12-16 13:24:00
8923,ADE-348904,MT,2016-04-05 22:51:00,2016-04-09 09:45:00
8924,ADE-425431,DS,2018-01-29 20:30:00,2018-01-31 11:27:00


#### 1.1.2  Critical Capacity Data
Key data to hospital Capacity Analysis

In [7]:
mt_beds = 13 #Current Number of Beds Available in Maternity
ds_beds = 4  #Current Number of Beds Available in Delivery Suite

### 1.4 Profile Length of Stay

#### 1.4.2 Profile of Length of Stay by Ward

In [8]:
ideal_df.groupby('Ward_Code').describe()

LOS                                                        \
          count                       mean                        std   
Ward_Code                                                               
DS         2339  0 days 22:19:06.618212911  0 days 12:09:48.296435046   
ICU          81  1 days 06:26:45.185185185  0 days 20:24:22.634704846   
MT         5783  3 days 23:55:35.929448383  1 days 15:17:05.855380638   

                                                                               \
                       min              25%              50%              75%   
Ward_Code                                                                       
DS         0 days 00:01:00  0 days 14:33:00  0 days 22:15:00  1 days 04:30:00   
ICU        0 days 02:34:00  0 days 17:53:00  0 days 23:59:00  1 days 15:17:00   
MT         0 days 01:30:00  3 days 04:25:00  3 days 20:38:00  4 days 18:36:00   

                            LOS_hrs             ...                          \
                        max   count       mean  ...         75%         max   
Ward_Code                                       ...                           
DS          5 days 22:19:00  2339.0  22.318505  ...   28.500000  142.316667   
ICU         6 days 15:52:00    81.0  30.445885  ...   39.283333  159.866667   
MT         34 days 19:50:00  5783.0  95.926647  ...  114.600000  835.833333   

            Count                                     
            count mean  std  min  25%  50%  75%  max  
Ward_Code                                             
DS         2339.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  
ICU          81.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  
MT         5784.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  

[3 rows x 24 columns]

#### 1.4.2 Profile of Length of Stay Maternity

In [9]:
ideal_df['LOS'][ideal_df['Ward_Code'] == 'MT'].value_counts(bins=10, normalize=True)*100

(3 days 12:56:00, 7 days 00:22:00]              65.819502
(0 days 00:39:56.399999999, 3 days 12:56:00]    32.365145
(7 days 00:22:00, 10 days 11:48:00]              1.210235
(10 days 11:48:00, 13 days 23:14:00]             0.311203
(13 days 23:14:00, 17 days 10:40:00]             0.103734
(17 days 10:40:00, 20 days 22:06:00]             0.069156
(31 days 08:24:00, 34 days 19:50:00]             0.034578
(27 days 20:58:00, 31 days 08:24:00]             0.034578
(24 days 09:32:00, 27 days 20:58:00]             0.017289
(20 days 22:06:00, 24 days 09:32:00]             0.017289
Name: LOS, dtype: float64

#### 1.4.3 Profile of Length of Stay Maternity

In [10]:
ideal_df['LOS'][ideal_df['Ward_Code'] == 'DS'].value_counts(bins=10, normalize=True)*100

(0 days 14:14:48, 1 days 04:28:36]                50.705430
(-1 days +23:52:27.719999999, 0 days 14:14:48]    23.899102
(1 days 04:28:36, 1 days 18:42:24]                21.633177
(1 days 18:42:24, 2 days 08:56:12]                 2.607952
(2 days 08:56:12, 2 days 23:10:00]                 0.726806
(2 days 23:10:00, 3 days 13:23:48]                 0.213767
(3 days 13:23:48, 4 days 03:37:36]                 0.128260
(5 days 08:05:12, 5 days 22:19:00]                 0.042753
(4 days 03:37:36, 4 days 17:51:24]                 0.042753
(4 days 17:51:24, 5 days 08:05:12]                 0.000000
Name: LOS, dtype: float64

### 1.5 Profile - Arrival Rate

In [158]:
arrival_df

,Date,Event,Link,Ward_Code,Start,End,LOS,LOS_hrs,Count,Prev_Flow,Full_Flow_Code,Day_of_Week,Month,Year,Hour_of_Day,Interarrival_Time
5,2013-06-27,NORMAL,ADE-282775,MT,2013-06-27 16:00:00,2013-07-01 11:05:00,3 days 19:05:00,91.0833,1,IN,IN-MT,Thursday,June,2013,16,200.0
4,2013-06-27,NORMAL,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00,81.5167,1,IN,IN-MT,Thursday,June,2013,23,474.0
9,2013-06-28,NORMAL,ADE-282886,MT,2013-06-28 06:30:00,2013-07-02 10:09:00,4 days 03:39:00,99.65,1,IN,IN-MT,Friday,June,2013,6,396.0
8,2013-06-28,NORMAL,ADE-273071,MT,2013-06-28 12:27:00,2013-07-02 11:05:00,3 days 22:38:00,94.6333,1,IN,IN-MT,Friday,June,2013,12,357.0
7,2013-06-28,NORMAL,ADE-272854,MT,2013-06-28 21:12:00,2013-07-01 10:37:00,2 days 13:25:00,61.4167,1,IN,IN-MT,Friday,June,2013,21,525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8161,2020-07-27,NORMAL,ADE-506202,MT,2020-07-27 13:43:00,2020-07-29 15:58:00,2 days 02:15:00,50.25,1,IN,IN-MT,Monday,July,2020,13,2722.0
8159,2020-07-27,NORMAL,ADE-501132,MT,2020-07-27 22:32:00,2020-07-31 10:04:00,3 days 11:32:00,83.5333,1,IN,IN-MT,Monday,July,2020,22,529.0
8158,2020-07-28,NORMAL,ADE-510775,MT,2020-07-28 12:14:00,2020-07-28 17:15:00,0 days 05:01:00,5.01667,1,IN,IN-MT,Tuesday,July,2020,12,822.0
8157,2020-07-28,NORMAL,ADE-501113,MT,2020-07-28 15:29:00,2020-08-01 10:01:00,3 days 18:32:00,90.5333,1,IN,IN-MT,Tuesday,July,2020,15,195.0


### 1.6 Variability Analysis - Events of Significance
#### 1.6.1 Selected Events & Associated Dates

In [11]:
grouped_events_df.set_index('Date', inplace = True)
grouped_events_df.sample(frac=0.1)

,Event
Date,
2013-01-17,SCHOOL
2012-07-13,SCHOOL
2019-12-23,"SCHOOL,BEFOREXMAS"
2025-03-10,PUBLIC
2024-07-09,SCHOOL
...,...
2018-12-14,BEFOREXMAS
2017-07-13,SCHOOL
2012-07-14,SCHOOL


#### 1.6.2 Number of days in Dataset that these dates occur

In [19]:
grouped_events_df['Count'] = 1
grouped_events_df.groupby(['Event'], as_index=False)['Count'].count().sort_values('Count', ascending = False)

,Event,Count
13,SCHOOL,912
16,"SCHOOL,BEFOREXMAS",78
14,"SCHOOL,AFTERXMAS",63
1,B4EASTER,56
8,"PUBLIC,SCHOOL",47
2,BEFOREXMAS,46
3,"BEFOREXMAS,SCHOOL",44
5,PUBLIC,34
0,"AFTERXMAS,SCHOOL",30
7,"PUBLIC,EASTER",23


#### 1.6.3 Events of Significance brought back into idealised data

In [20]:
ideal_merged_df
#ideal_merged_df['Event'].notnull().sum()

,Date,Event,Link,Ward_Code,Start,End,LOS,LOS_hrs
0,2013-06-30,NORMAL,ADE-271165,MT,2013-06-30 22:21:00,2013-07-04 10:40:00,3 days 12:19:00,84.316667
1,2013-06-30,NORMAL,ADE-271746,MT,2013-06-30 12:00:00,2013-07-03 09:58:00,2 days 21:58:00,69.966667
2,2013-06-30,NORMAL,ADE-282352,MT,2013-06-30 13:00:00,2013-07-05 09:25:00,4 days 20:25:00,116.416667
3,2013-06-27,NORMAL,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00,21.666667
4,2013-06-27,NORMAL,ADE-271551,MT,2013-06-27 23:54:00,2013-07-01 09:25:00,3 days 09:31:00,81.516667
...,...,...,...,...,...,...,...,...
8199,2020-06-07,NORMAL,ADE-506055,MT,2020-06-07 17:48:00,2020-06-10 13:47:00,2 days 19:59:00,67.983333
8200,2020-06-07,NORMAL,ADE-506091,DS,2020-06-07 23:45:00,2020-06-08 10:30:00,0 days 10:45:00,10.750000
8201,2020-06-13,NORMAL,ADE-506671,DS,2020-06-13 20:00:00,2020-06-14 12:30:00,0 days 16:30:00,16.500000
8202,2020-07-12,SCHOOL,ADE-508881,DS,2020-07-12 21:30:00,2020-07-13 17:22:00,0 days 19:52:00,19.866667


## Objective 2
### 2.1 Current Capacity Requirements - Historical Data
#### 2.1.1 Maternity Transformed Data
Historical data of number of patients in Maternity at the same time

In [14]:
mt_count_df

,Event,Patient_Change,MT_Patients,Bed_Available
813,2013-06-27 16:00:00,1,1,True
12,2013-06-27 23:54:00,1,2,True
827,2013-06-28 06:30:00,1,3,True
5,2013-06-28 10:20:00,1,4,True
85,2013-06-28 12:27:00,1,5,True
...,...,...,...,...
4741,2020-07-29 15:58:00,-1,3,True
2274,2020-07-29 23:45:00,1,4,True
7819,2020-07-30 09:52:00,-1,3,True
6456,2020-07-31 10:04:00,-1,2,True


#### 2.1.2 Maternity Occupancy Profile

In [15]:
mt_count_df['MT_Patients'].describe()

count    11957.000000
mean         8.669482
std          2.515389
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         15.000000
Name: MT_Patients, dtype: float64

#### 2.1.3 Delivery Suite Transformed Data

In [16]:
ds_count_df

,Event,Patient_Change,DS_Patients,Bed_Available
4,2013-06-27 12:40:00,1,1,True
4,2013-06-28 10:20:00,-1,0,True
44,2013-06-29 03:00:00,1,1,True
44,2013-06-30 12:00:00,-1,0,True
899,2013-07-03 21:04:00,1,1,True
...,...,...,...,...
4757,2020-07-24 19:30:00,-1,0,True
418,2020-07-24 19:47:00,1,1,True
7818,2020-07-25 16:21:00,1,2,True
418,2020-07-25 20:08:00,-1,1,True


#### 2.1.4 Delivery Suite Occupancy Profile

In [17]:
ds_count_df['DS_Patients'].describe()

count    4720.000000
mean        1.388136
std         1.022002
min         0.000000
25%         1.000000
50%         1.000000
75%         2.000000
max         5.000000
Name: DS_Patients, dtype: float64

### 2.2 Current Capacity Requirements - Idealised Historical Data
**Create new ideal patient flow records with existing data and clean the dataset**

In an ideal scenario, rather than sending patients to St Clares, St Helens and St Catherines they are sent to Maternity.  This process adjusts the actual data to what the ideal data would be.

As DPS is not relevant in this study we would also like to remove these entries from the dataset.

Once the data has been converted to ideal there are entries where the patient is transferred to the same room they left.  To clean the data we need to find the start date of when they first arrvied in the room and the end date when they left.

Once found we need to remove the duplicate entries and only show the first start date and last end date.

In [18]:
ideal_df

,Link,Ward_Code,Start,End,LOS,LOS_hrs,Date,Count,Prev_Flow
4,ADE-271206,DS,2013-06-27 12:40:00,2013-06-28 10:20:00,0 days 21:40:00,21.666667,2013-06-27,1,IN
14,ADE-271568,DS,2013-07-11 07:16:00,2013-07-12 13:59:00,1 days 06:43:00,30.716667,2013-07-11,1,IN
20,ADE-271570,DS,2013-07-12 10:15:00,2013-07-13 11:43:00,1 days 01:28:00,25.466667,2013-07-12,1,IN
25,ADE-271572,DS,2013-07-29 23:44:00,2013-07-30 22:52:00,0 days 23:08:00,23.133333,2013-07-29,1,IN
33,ADE-271691,DS,2013-07-10 06:00:00,2013-07-11 10:38:00,1 days 04:38:00,28.633333,2013-07-10,1,IN
...,...,...,...,...,...,...,...,...,...
773,ADE-451883,ICU,2018-11-03 14:05:00,2018-11-04 16:53:00,1 days 02:48:00,26.800000,2018-11-03,1,IN-MT
568,ADE-495564,ICU,2020-02-23 06:49:00,2020-02-25 11:37:00,2 days 04:48:00,52.800000,2020-02-23,1,IN-MT
7400,ADE-496923,ICU,2020-02-21 03:37:00,2020-02-23 13:10:00,2 days 09:33:00,57.550000,2020-02-21,1,IN-MT
2368,ADE-501424,ICU,2020-05-21 21:05:00,2020-05-28 12:57:00,6 days 15:52:00,159.866667,2020-05-21,1,IN-MT
